In [3]:
import pandas as pd
import requests
import time
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr, spearmanr

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

In [4]:
! pip install Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.6 MB/s eta 0:00:0000:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.38
    Uninstalling SQLAlchemy-2.0.38:
      Successfully uninstalled SQLAlchemy-2.0.38
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


# Download Data 

In [5]:
def get_dataset_rows(dataset, config, split, offset=0, limit=100):
    url = f"https://datasets-server.huggingface.co/rows?dataset={dataset}&config={config}&split={split}&offset={offset}&limit={limit}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("rows", [])
    else:
        print(f"Erreur HTTP {response.status_code}")
        return []

dataset = "cnamuangtoun/resume-job-description-fit"
config = "default"
split = "train"
limit = 100
offset = 0

all_rows = []

while True:
    rows = get_dataset_rows(dataset, config, split, offset, limit)
    if not rows:
        break  # Stop if no more data
    all_rows.extend(rows)
    print(f"Fetched {len(rows)} rows (total so far: {len(all_rows)})")
    offset += limit
    time.sleep(0.5)  # Respect API rate limits

print(f"Total rows collected: {len(all_rows)}")



Fetched 100 rows (total so far: 100)
Fetched 100 rows (total so far: 200)
Fetched 100 rows (total so far: 300)
Fetched 100 rows (total so far: 400)
Fetched 100 rows (total so far: 500)
Fetched 100 rows (total so far: 600)
Fetched 100 rows (total so far: 700)
Fetched 100 rows (total so far: 800)
Fetched 100 rows (total so far: 900)
Fetched 100 rows (total so far: 1000)
Fetched 100 rows (total so far: 1100)
Fetched 100 rows (total so far: 1200)
Fetched 100 rows (total so far: 1300)
Fetched 100 rows (total so far: 1400)
Fetched 100 rows (total so far: 1500)
Fetched 100 rows (total so far: 1600)
Fetched 100 rows (total so far: 1700)
Fetched 100 rows (total so far: 1800)
Fetched 100 rows (total so far: 1900)
Fetched 100 rows (total so far: 2000)
Fetched 100 rows (total so far: 2100)
Fetched 100 rows (total so far: 2200)
Fetched 100 rows (total so far: 2300)
Fetched 100 rows (total so far: 2400)
Fetched 100 rows (total so far: 2500)
Fetched 100 rows (total so far: 2600)
Fetched 100 rows (tot

In [12]:

offset = 3500
while True:
    rows = get_dataset_rows(dataset, config, split, offset, limit)
    if not rows:
        break  # Stop if no more data
    all_rows.extend(rows)
    print(f"Fetched {len(rows)} rows (total so far: {len(all_rows)})")
    offset += limit
    time.sleep(6)  # Respect API rate limits


print(f"Total rows collected: {len(all_rows)}")
data=all_rows

Fetched 100 rows (total so far: 3600)
Fetched 100 rows (total so far: 3700)
Fetched 100 rows (total so far: 3800)
Fetched 100 rows (total so far: 3900)
Fetched 100 rows (total so far: 4000)
Fetched 100 rows (total so far: 4100)
Fetched 100 rows (total so far: 4200)
Fetched 100 rows (total so far: 4300)
Fetched 100 rows (total so far: 4400)
Fetched 100 rows (total so far: 4500)
Fetched 100 rows (total so far: 4600)
Fetched 100 rows (total so far: 4700)
Fetched 100 rows (total so far: 4800)
Fetched 100 rows (total so far: 4900)
Fetched 100 rows (total so far: 5000)
Fetched 100 rows (total so far: 5100)
Fetched 100 rows (total so far: 5200)
Fetched 100 rows (total so far: 5300)
Fetched 100 rows (total so far: 5400)
Fetched 100 rows (total so far: 5500)
Fetched 100 rows (total so far: 5600)
Fetched 100 rows (total so far: 5700)
Fetched 100 rows (total so far: 5800)
Fetched 100 rows (total so far: 5900)
Fetched 100 rows (total so far: 6000)
Fetched 100 rows (total so far: 6100)
Fetched 100 

# Data Analysis

Converting it to Dataframe 

In [13]:
if data:
    job_descriptions = []
    resume_texts = []
    fits = []

    for row in data:
        row=row['row']
        job_descriptions.append(row.get('job_description_text', ''))  # Handle missing keys
        resume_texts.append(row.get('resume_text', ''))
        fits.append(row.get('label', ''))

    df = pd.DataFrame({
        'job_description_text': job_descriptions,
        'resume_text': resume_texts,
        'label': fits
    })

df.head()
len(df)

6241

Replace labels with numbers

In [14]:
df['label'] = df['label'].replace({'No Fit': 0,'Potential Fit':2,'Good Fit':1})
df.head()


/tmp/ipykernel_31/3045072375.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'No Fit': 0,'Potential Fit':2,'Good Fit':1})


,job_description_text,resume_text,label
0,Net2Source Inc. is an award-winning total work...,SummaryHighly motivated Sales Associate with e...,0
1,At Salas OBrien we tell our clients that were ...,Professional SummaryCurrently working with Cat...,0
2,Schweitzer Engineering Laboratories (SEL) Infr...,SummaryI started my construction career in Jun...,0
3,"Mizick Miller & Company, Inc. is looking for a...",SummaryCertified Electrical Foremanwith thirte...,0
4,Life at Capgemini\nCapgemini supports all aspe...,SummaryWith extensive experience in business/r...,0


1. # **Working on subset first**

In [15]:

df_subset=pd.concat([df[0:100],df[3300:3400],df[6000:6100]])


In [16]:
df_subset.head()

,job_description_text,resume_text,label
0,Net2Source Inc. is an award-winning total work...,SummaryHighly motivated Sales Associate with e...,0
1,At Salas OBrien we tell our clients that were ...,Professional SummaryCurrently working with Cat...,0
2,Schweitzer Engineering Laboratories (SEL) Infr...,SummaryI started my construction career in Jun...,0
3,"Mizick Miller & Company, Inc. is looking for a...",SummaryCertified Electrical Foremanwith thirte...,0
4,Life at Capgemini\nCapgemini supports all aspe...,SummaryWith extensive experience in business/r...,0


Using pretrained SBERT

In [17]:
df_subset.head()

,job_description_text,resume_text,label
0,Net2Source Inc. is an award-winning total work...,SummaryHighly motivated Sales Associate with e...,0
1,At Salas OBrien we tell our clients that were ...,Professional SummaryCurrently working with Cat...,0
2,Schweitzer Engineering Laboratories (SEL) Infr...,SummaryI started my construction career in Jun...,0
3,"Mizick Miller & Company, Inc. is looking for a...",SummaryCertified Electrical Foremanwith thirte...,0
4,Life at Capgemini\nCapgemini supports all aspe...,SummaryWith extensive experience in business/r...,0


In [18]:
!wandb login 0843e8b04fd5c4cb0a5cbe12d5231a3f02f9bff3


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Finetune SBERT on this subset

In [19]:
# 1. Mapping les labels à un score de similarité entre 0 et 1
label_map = {0: 0.0, 2: 0.5, 1: 1.0}  # Tu peux ajuster le 0.5 si tu veux pondérer différemment

# 2. Convertir les lignes du DataFrame en InputExample
examples = [
    InputExample(
        texts=[row["resume_text"], row["job_description_text"]],
        label=label_map[int(row["label"])]
    )
    for _, row in df_subset.iterrows()
]

# 3. Split train / validation
train_examples, dev_examples = train_test_split(examples, test_size=0.2, random_state=42)

# 4. DataLoaders
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
dev_dataloader = DataLoader(dev_examples, shuffle=False, batch_size=16)

# 5. Charger le modèle SBERT pré-entraîné
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 6. Définir la loss de type similarité cosinus
train_loss = losses.CosineSimilarityLoss(model=model)

# 7. Définir un évaluateur basé sur la similarité supervisée
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_examples, name='dev-similarity')

# 8. Dossier de sortie pour le modèle fine-tuné

"""from transformers import TrainerCallback
import os

# Désactiver wandb
os.environ["WANDB_MODE"] = "offline"""

# 9. Entraînement
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=4,
    evaluation_steps=500,
    warmup_steps=100,
    save_best_model=True
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: zina-aloui (ines_ecl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,Dev-similarity Pearson Cosine,Dev-similarity Spearman Cosine
15,No log,No log,0.399560,0.461891
30,No log,No log,0.444924,0.497853
45,No log,No log,0.518116,0.518082
60,No log,No log,0.586794,0.565283


In [20]:
# Define the new label mapping
label_remap = {0.0: 0, 0.5: 2, 1.0: 1}

# Apply the mapping and print
for ex in dev_examples:
    original = ex.label
    mapped = label_remap[original]
mapped_labels = [label_remap[ex.label] for ex in dev_examples]


In [ ]:
Model evaluation

In [21]:
def compute_cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

true_labels = []
pred_similarities = []

# test_subset
for example in dev_examples:  # list of InputExample(texts=[resume, job], label=0/0.5/1)
    emb1, emb2 = model.encode(example.texts)
    sim = compute_cosine_similarity(emb1, emb2)
    pred_similarities.append(sim)
    true_labels.append(example.label)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Compute correlation between predicted and true values

In [23]:
print("Pearson:", pearsonr(pred_similarities, mapped_labels))
print("Spearman:", spearmanr(pred_similarities, mapped_labels))


Pearson: PearsonRResult(statistic=0.5471135966040306, pvalue=6.088716617658881e-06)
Spearman: SignificanceResult(statistic=0.5384292912506924, pvalue=9.101908927821827e-06)


classification results

In [24]:
def classify_similarity(sim):
    if sim >= 0.7:
        return 1
    elif sim >= 0.4:
        return 2
    else:
        return 0

pred_classes = [classify_similarity(s) for s in pred_similarities]


In [25]:
print(classification_report(mapped_labels, pred_classes, digits=3))


              precision    recall  f1-score   support

           0      0.783     0.818     0.800        22
           1      0.714     0.227     0.345        22
           2      0.400     0.750     0.522        16

    accuracy                          0.583        60
   macro avg      0.632     0.598     0.556        60
weighted avg      0.656     0.583     0.559        60



In [26]:
# overall accuracy of the subset
overall_accuracy = accuracy_score(mapped_labels, pred_classes)
print(f"Overall accuracy of subset: {overall_accuracy:.3f}")


Overall accuracy of subset: 0.583


# **working on the Dataset**

In [28]:
# 1. Mapping les labels à un score de similarité entre 0 et 1
label_map = {0: 0.0, 2: 0.5, 1: 1.0}  # Tu peux ajuster le 0.5 si tu veux pondérer différemment

# 2. Convertir les lignes du DataFrame en InputExample
examples_df = [
    InputExample(
        texts=[row["resume_text"], row["job_description_text"]],
        label=label_map[int(row["label"])]
    )
    for _, row in df.iterrows()
]

# 3. Split train / validation
train_examples_df, dev_examples_df = train_test_split(examples_df, test_size=0.2, random_state=42)

# 4. DataLoaders
train_dataloader_df = DataLoader(train_examples_df, shuffle=True, batch_size=16)
dev_dataloader_df = DataLoader(dev_examples_df, shuffle=False, batch_size=16)

# 5. Charger le modèle SBERT pré-entraîné
model_df = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 6. Définir la loss de type similarité cosinus
train_loss_df = losses.CosineSimilarityLoss(model=model_df)

# 7. Définir un évaluateur basé sur la similarité supervisée
evaluator_df = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_examples_df, name='dev-similarity')

# 8. Dossier de sortie pour le modèle fine-tuné

"""from transformers import TrainerCallback
import os

# Désactiver wandb
os.environ["WANDB_MODE"] = "offline"""

# 9. Entraînement
model_df.fit(
    train_objectives=[(train_dataloader_df, train_loss_df)],
    evaluator=evaluator_df,
    epochs=4,
    evaluation_steps=500,
    warmup_steps=100,
    save_best_model=True
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Dev-similarity Pearson Cosine,Dev-similarity Spearman Cosine
312,No log,No log,0.588264,0.590523
500,0.124800,No log,0.644714,0.640355
624,0.124800,No log,0.661579,0.652916
936,0.124800,No log,0.695810,0.678815
1000,0.084900,No log,0.698423,0.683593
1248,0.084900,No log,0.706958,0.691944


Model evaluation

In [38]:
def compute_cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

true_labels_df = []
pred_similarities_df = []

for example in dev_examples_df:  # list of InputExample(texts=[resume, job], label=0/0.5/1)
    emb1, emb2 = model_df.encode(example.texts)
    sim = compute_cosine_similarity(emb1, emb2)
    pred_similarities_df.append(sim)
    true_labels_df.append(example.label)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
# Define the new label mapping
label_remap = {0.0: 0, 0.5: 2, 1.0: 1}

# Apply the mapping and print
# for ex in dev_examples_df:
#     original = ex.label
#     mapped = label_remap[original]
pred_classes_df = [classify_similarity(s) for s in pred_similarities_df]

mapped_labels_df_all = [label_remap[ex.label] for ex in dev_examples_df]


In [40]:

print("Pearson:", pearsonr(pred_similarities_df, mapped_labels_df_all))
print("Spearman:", spearmanr(pred_similarities_df, mapped_labels_df_all))


Pearson: PearsonRResult(statistic=0.4258088103753144, pvalue=3.565404175828347e-56)
Spearman: SignificanceResult(statistic=0.5077940203725082, pvalue=7.889828804712564e-83)


In [41]:

print(classification_report(pred_classes_df,mapped_labels_df_all, digits=3))


              precision    recall  f1-score   support

           0      0.880     0.736     0.801       787
           1      0.396     0.845     0.539       142
           2      0.361     0.325     0.342       320

    accuracy                          0.643      1249
   macro avg      0.546     0.635     0.561      1249
weighted avg      0.692     0.643     0.654      1249



In [43]:
# overall accuracy of the dataset

overall_accuracy = accuracy_score(mapped_labels_df_all, pred_classes_df)
print(f"Overall accuracy of Dataset: {overall_accuracy:.3f}")


Overall accuracy of Dataset: 0.643


In [45]:
model_df.save("finetuned-sbert-v2")

In [47]:
!zip -r finetuned-sbert-v2.zip finetuned-sbert-v2


  adding: finetuned-sbert-v2/ (stored 0%)
  adding: finetuned-sbert-v2/tokenizer_config.json (deflated 73%)
  adding: finetuned-sbert-v2/sentence_bert_config.json (deflated 4%)
  adding: finetuned-sbert-v2/2_Normalize/ (stored 0%)
  adding: finetuned-sbert-v2/1_Pooling/ (stored 0%)
  adding: finetuned-sbert-v2/1_Pooling/config.json (deflated 57%)
  adding: finetuned-sbert-v2/config.json (deflated 48%)
  adding: finetuned-sbert-v2/README.md (deflated 68%)
  adding: finetuned-sbert-v2/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: finetuned-sbert-v2/vocab.txt (deflated 53%)
  adding: finetuned-sbert-v2/special_tokens_map.json (deflated 80%)
  adding: finetuned-sbert-v2/modules.json (deflated 62%)
  adding: finetuned-sbert-v2/tokenizer.json (deflated 71%)
  adding: finetuned-sbert-v2/config_sentence_transformers.json (deflated 35%)


In [48]:
from IPython.display import FileLink
FileLink("finetuned-sbert-v2.zip")


/kaggle/working/finetuned-sbert-v2.zip

wandb: 
wandb: 🚀 View run checkpoints/model at: https://wandb.ai/ines_ecl/sentence-transformers/runs/3xsq4koy
wandb: Find logs at: wandb/run-20250508_174402-3xsq4koy/logs
